<a href="https://colab.research.google.com/github/indranildchandra/Devanagri_Character_Recognition_Classifier/blob/master/src/Devanagri_Character_Recognition_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#upload data.zip file for training

Saving data.csv.zip to data.csv.zip
User uploaded file "data.csv.zip" with length 49541075 bytes


In [0]:
!unzip "data.csv.zip"

Archive:  data.csv.zip
  inflating: data.csv                


In [0]:
!ls -ltr

In [0]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [0]:
data = Nune
df = pd.read_csv('data.csv')
df.head()

,pixel_0000,pixel_0001,pixel_0002,pixel_0003,pixel_0004,pixel_0005,pixel_0006,pixel_0007,pixel_0008,pixel_0009,pixel_0010,pixel_0011,pixel_0012,pixel_0013,pixel_0014,pixel_0015,pixel_0016,pixel_0017,pixel_0018,pixel_0019,pixel_0020,pixel_0021,pixel_0022,pixel_0023,pixel_0024,pixel_0025,pixel_0026,pixel_0027,pixel_0028,pixel_0029,pixel_0030,pixel_0031,pixel_0032,pixel_0033,pixel_0034,pixel_0035,pixel_0036,pixel_0037,pixel_0038,pixel_0039,...,pixel_0985,pixel_0986,pixel_0987,pixel_0988,pixel_0989,pixel_0990,pixel_0991,pixel_0992,pixel_0993,pixel_0994,pixel_0995,pixel_0996,pixel_0997,pixel_0998,pixel_0999,pixel_1000,pixel_1001,pixel_1002,pixel_1003,pixel_1004,pixel_1005,pixel_1006,pixel_1007,pixel_1008,pixel_1009,pixel_1010,pixel_1011,pixel_1012,pixel_1013,pixel_1014,pixel_1015,pixel_1016,pixel_1017,pixel_1018,pixel_1019,pixel_1020,pixel_1021,pixel_1022,pixel_1023,character
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,character_01_ka
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,character_01_ka
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,character_01_ka
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,character_01_ka
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,character_01_ka


In [0]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
x32 = [[i.reshape(32,32)] for i in x.values]
x32 = np.array(x32)

In [0]:
x32.shape

(92000, 1, 32, 32)

In [0]:
le = LabelEncoder()
target_y = le.fit_transform(y)
target_y = np_utils.to_categorical(target_y)

In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(x32, target_y, test_size=0.3)
xtrain = xtrain/255
xtest = xtest/255

In [0]:
model = Sequential()

#input
K.set_image_dim_ordering('th')
model.add(Conv2D(128, (5, 5), input_shape=(1, 32, 32), activation='relu'))

# model building (hidden layer)
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.33))

# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(15, (3, 3), activation='relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dense(80, activation='relu'))


#output
model.add(Dense(46, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 128, 28, 28)       3328      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 128, 9, 9)         0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128, 9, 9)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               1327232   
_________________________________________________________________
dense_17 (Dense)             (None, 46)                5934      
Total params: 1,336,494
Trainable params: 1,336,494
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time

model.fit(xtrain, ytrain, epochs=50, validation_data=(xtest, ytest), batch_size=200)


Train on 64400 samples, validate on 27600 samples
Epoch 1/50
64400/64400 [==============================] - 5s 85us/step - loss: 0.9417 - acc: 0.7507 - val_loss: 0.3512 - val_acc: 0.8995
Epoch 2/50
64400/64400 [==============================] - 5s 74us/step - loss: 0.2746 - acc: 0.9210 - val_loss: 0.2239 - val_acc: 0.9356
Epoch 3/50
64400/64400 [==============================] - 5s 74us/step - loss: 0.1848 - acc: 0.9456 - val_loss: 0.1878 - val_acc: 0.9442
Epoch 4/50
64400/64400 [==============================] - 5s 74us/step - loss: 0.1431 - acc: 0.9565 - val_loss: 0.1542 - val_acc: 0.9540
Epoch 5/50
64400/64400 [==============================] - 5s 74us/step - loss: 0.1090 - acc: 0.9677 - val_loss: 0.1420 - val_acc: 0.9568
Epoch 6/50
64400/64400 [==============================] - 5s 75us/step - loss: 0.0907 - acc: 0.9724 - val_loss: 0.1358 - val_acc: 0.9605
Epoch 7/50
64400/64400 [==============================] - 5s 74us/step - loss: 0.0725 - acc: 0.9781 - val_loss: 0.1316 - val_acc

In [0]:
scores = model.evaluate(xtest, ytest)
print("Model Error: %.2f%%" % (100-scores[1]*100), "acc: " + str(scores[1]))

27600/27600 [==============================] - 2s 63us/step
Large CNN Error: 2.57% acc: 0.9743478260869565
